1.Data cleaning and integration

Import basic libraries

In [58]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
from scipy.spatial import cKDTree
from numpy import radians, sin, cos, sqrt, arctan2
import os
import pygeohash as pgh
import plotly.express as px
from project.functions import helper_functions as hf

In [3]:
os.chdir('C:/Users/wissam_T/Desktop/5th/2nd semester/dm/h.w 1/project')

Read and Preview the Daily Rent Data

In [16]:
df = pd.read_parquet("datasets/daily-rent-sampled.parquet")
df.head(), df.columns

(            ride_id rideable_type          started_at            ended_at  \
 0  6A6067934F2E1EC9  classic_bike 2024-05-04 14:27:37 2024-05-04 14:37:52   
 1  FF5D6FE256D4DE34  classic_bike 2024-05-04 14:00:53 2024-05-04 15:00:25   
 2  4A64EABCDCB7506A  classic_bike 2024-05-04 15:12:21 2024-05-04 15:27:29   
 3  0F7347CF6409CEB7  classic_bike 2024-05-04 14:49:59 2024-05-04 14:54:11   
 4  2022D12F38B197C3  classic_bike 2024-05-04 19:23:20 2024-05-04 19:41:35   
 
             start_station_name start_station_id  \
 0               14th & G St NW            31238   
 1    Jefferson Dr & 14th St SW            31247   
 2                1st & M St NE            31603   
 3           Jefferson Memorial            31249   
 4  New Hampshire Ave & T St NW            31229   
 
                             end_station_name end_station_id  start_lat  \
 0                         Jefferson Memorial          31249  38.898164   
 1  7th & F St NW / National Portrait Gallery          31232  38.8

Cleaning and Preparation of Trip Data for Analysis

In [17]:
# 1. Remove duplicate trips based on ride_id
before = len(df)
df_cleaned = df.drop_duplicates(subset=['ride_id'])
after = len(df_cleaned)
print(f"[Step 1] Duplicates removed: {before - after} | Rows after: {after}")

# 2. Ensure correct datetime types for started_at and ended_at
df_cleaned['started_at'] = pd.to_datetime(df_cleaned['started_at'])
df_cleaned['ended_at'] = pd.to_datetime(df_cleaned['ended_at'])

# 3. Remove trips where end time is before start time
before = len(df_cleaned)
df_cleaned = df_cleaned[df_cleaned['ended_at'] > df_cleaned['started_at']].copy()
after = len(df_cleaned)
print(f"[Step 3] Rows removed (end before start): {before - after} | Rows after: {after}")

# Step 4: Calculate trip duration in minutes
df_cleaned['duration_min'] = (df_cleaned['ended_at'] - df_cleaned['started_at']).dt.total_seconds() / 60

before = len(df_cleaned)

# Only remove trips shorter than 1 minute
unreasonable_trips = df_cleaned[df_cleaned['duration_min'] < 1].copy()

df_cleaned = df_cleaned[df_cleaned['duration_min'] >= 1].copy()

after = len(df_cleaned)

print(f"[Step 4] Rows removed (duration < 1 min): {before - after} | Rows after: {after}")
print(f"Number of removed trips: {len(unreasonable_trips)}")

# Show some examples
print(unreasonable_trips[['ride_id', 'duration_min', 'started_at', 'ended_at']].head(10))

# 5. Clean 'member_casual' values
valid_values = ['member', 'casual']
df_cleaned = df_cleaned[df_cleaned['member_casual'].isin(valid_values)].copy()
print(f"[Step 5] Rows after cleaning 'member_casual': {len(df_cleaned)}")

# 6. Reset index for clarity
df_cleaned = df_cleaned.reset_index(drop=True)
print(f"[Step 6] Index reset. Final row count: {len(df_cleaned)}")

# 7. Show count of null values per column
print("[Step 7] Null values per column:")
print(df_cleaned.isnull().sum())

# 8. Create geometry columns for trip start and end
df_cleaned['geometry_start'] = df_cleaned.apply(lambda row: Point(row['start_lng'], row['start_lat']), axis=1)
df_cleaned['geometry_end'] = df_cleaned.apply(lambda row: Point(row['end_lng'], row['end_lat']), axis=1)
print("[Step 8] Geometry columns for start and end created.")

# 9. Create GeoDataFrames for start and end points
gdf_start = gpd.GeoDataFrame(df_cleaned.copy(), geometry='geometry_start', crs="EPSG:4326")
gdf_end = gpd.GeoDataFrame(df_cleaned.copy(), geometry='geometry_end', crs="EPSG:4326")
print(f"[Step 9] gdf_start CRS: {gdf_start.crs} | gdf_end CRS: {gdf_end.crs}")


# 10. Clean and format station information columns:
# - Drop raw latitude and longitude columns (now replaced by geometry)
# - Convert station ID columns to numeric types, handling errors
# - Replace missing station names with 'Unknown'
# - Replace missing station IDs with -1 for consistency
cols_to_drop = ['start_lat', 'start_lng', 'end_lat', 'end_lng']
df_cleaned = df_cleaned.drop(columns=cols_to_drop)
df_cleaned['start_station_id'] = pd.to_numeric(df_cleaned['start_station_id'], errors='coerce').astype('Int64')
df_cleaned['end_station_id'] = pd.to_numeric(df_cleaned['end_station_id'], errors='coerce').astype('Int64')
df_cleaned['start_station_name'].fillna('Unknown', inplace=True)
df_cleaned['start_station_id'].fillna(-1, inplace=True)
df_cleaned['end_station_name'].fillna('Unknown', inplace=True)
df_cleaned['end_station_id'].fillna(-1, inplace=True)
print("Dropped columns:", cols_to_drop)
display(df_cleaned.head(20))
print("[Step 10] Null values per column:")
print(df_cleaned.isnull().sum())

[Step 1] Duplicates removed: 26 | Rows after: 2018430


C:\Users\wissam_T\AppData\Local\Temp\ipykernel_12388\3389113924.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['started_at'] = pd.to_datetime(df_cleaned['started_at'])
C:\Users\wissam_T\AppData\Local\Temp\ipykernel_12388\3389113924.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['ended_at'] = pd.to_datetime(df_cleaned['ended_at'])


[Step 3] Rows removed (end before start): 335 | Rows after: 2018095
[Step 4] Rows removed (duration < 1 min): 37863 | Rows after: 1980232
Number of removed trips: 37863
              ride_id  duration_min          started_at            ended_at
66   6137459834E41601      0.750000 2024-05-04 10:44:06 2024-05-04 10:44:51
166  9B27B6AAF9745304      0.933333 2024-05-04 15:41:07 2024-05-04 15:42:03
191  6353C1C863DB5750      0.083333 2024-05-04 09:53:02 2024-05-04 09:53:07
244  3EEF01728EDCFA27      0.066667 2024-05-04 09:09:00 2024-05-04 09:09:04
302  8306D54BA90B92E0      0.300000 2024-05-04 08:23:52 2024-05-04 08:24:10
337  8A9DC18F1A112B0B      0.683333 2024-05-04 12:06:42 2024-05-04 12:07:23
379  1698C6AB84A550BD      0.633333 2024-05-04 21:28:59 2024-05-04 21:29:37
439  67B99ABD3485F0D3      0.183333 2024-05-04 11:53:01 2024-05-04 11:53:12
495  CF3AC70459088588      0.683333 2024-05-04 09:49:36 2024-05-04 09:50:17
505  10E1C5BA8558DD62      0.233333 2024-05-04 12:05:36 2024-05-04 12:0

C:\Users\wissam_T\AppData\Local\Temp\ipykernel_12388\3389113924.py:68: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_cleaned['start_station_name'].fillna('Unknown', inplace=True)
C:\Users\wissam_T\AppData\Local\Temp\ipykernel_12388\3389113924.py:69: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always beh

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,member_casual,duration_min,geometry_start,geometry_end
0,6A6067934F2E1EC9,classic_bike,2024-05-04 14:27:37,2024-05-04 14:37:52,14th & G St NW,31238,Jefferson Memorial,31249,casual,10.250000,POINT (-77.03189760446548 38.89816396413188),POINT (-77.037413 38.879819)
1,FF5D6FE256D4DE34,classic_bike,2024-05-04 14:00:53,2024-05-04 15:00:25,Jefferson Dr & 14th St SW,31247,7th & F St NW / National Portrait Gallery,31232,casual,59.533333,POINT (-77.032427 38.8885525),POINT (-77.02219069004059 38.89728304853854)
2,4A64EABCDCB7506A,classic_bike,2024-05-04 15:12:21,2024-05-04 15:27:29,1st & M St NE,31603,4th & M St SW,31108,casual,15.133333,POINT (-77.005483 38.905697),POINT (-77.01777443289757 38.87682269362588)
3,0F7347CF6409CEB7,classic_bike,2024-05-04 14:49:59,2024-05-04 14:54:11,Jefferson Memorial,31249,Hains Point/Buckeye & Ohio Dr SW,31273,casual,4.200000,POINT (-77.037413 38.879819),POINT (-77.03023 38.878433)
4,2022D12F38B197C3,classic_bike,2024-05-04 19:23:20,2024-05-04 19:41:35,New Hampshire Ave & T St NW,31229,14th & G St NW,31238,casual,18.250000,POINT (-77.038252 38.915544),POINT (-77.03189760446548 38.89816396413188)
5,7DF015F77985083F,classic_bike,2024-05-04 17:44:03,2024-05-04 17:52:50,19th St & Constitution Ave NW,31235,17th & G St NW,31277,casual,8.783333,POINT (-77.04307436943054 38.89221235271478),POINT (-77.039643 38.898301)
6,0D2868E393144007,classic_bike,2024-05-04 15:05:02,2024-05-04 15:12:50,21st St & G st NW,31328,New Hampshire Ave & Ward Pl NW,31212,casual,7.800000,POINT (-77.04691261053085 38.89830173667249),POINT (-77.0473176240921 38.90571079190728)
7,1AA5CA4CA8265244,classic_bike,2024-05-04 17:58:50,2024-05-04 18:06:08,Eckington Pl & Q St NE,31505,MBT & Bryant St NE,31531,casual,7.300000,POINT (-77.00495 38.910972),POINT (-76.9960230588913 38.92240364115446)
8,F7D197C50FC5B8AF,classic_bike,2024-05-04 10:56:39,2024-05-04 11:27:01,5th & F St NW,31620,Eastern Market Metro / Pennsylvania Ave & 8th ...,31613,casual,30.366667,POINT (-77.019347 38.897222),POINT (-76.99523448944092 38.88409514578027)
9,13EC70FF77A305E3,classic_bike,2024-05-04 22:40:59,2024-05-04 22:50:14,14th & Q St NW,31327,New Jersey Ave & N St NW/Dunbar HS,31636,casual,9.250000,POINT (-77.03188 38.910674),POINT (-77.01536044478416 38.90733256111448)


[Step 10] Null values per column:
ride_id               0
rideable_type         0
started_at            0
ended_at              0
start_station_name    0
start_station_id      0
end_station_name      0
end_station_id        0
member_casual         0
duration_min          0
geometry_start        0
geometry_end          0
dtype: int64


In [18]:
df_cleaned.shape

(1980232, 12)

Cleaning and Preparing the Stations Data

In [19]:
# 1. Load the stations table
stations = pd.read_csv("datasets/stations.csv")
print(f"[Step 1] Loaded stations: {len(stations)} rows")

# 2. Remove duplicate stations by 'id'
before = len(stations)
stations = stations.drop_duplicates(subset=['id'])
after = len(stations)
print(f"[Step 2] Duplicates removed: {before - after} | Rows after: {after}")

# 3. Check for missing values
for col in ['id', 'name']:
    missing = stations[col].isnull().sum()
    print(f"[Step 3] Missing values in '{col}': {missing}")

# 4. Remove rows with missing values in critical columns
before = len(stations)
stations = stations.dropna(subset=['id', 'name'])
after = len(stations)
print(f"[Step 4] Rows removed due to missing critical info: {before - after} | Rows after: {after}")

# 5. Reset index for clarity
stations = stations.reset_index(drop=True)
print(f"[Step 5] Index reset. Final stations count: {len(stations)}")

# 6. Show cleaned stations table
display(stations.head())


[Step 1] Loaded stations: 821 rows
[Step 2] Duplicates removed: 17 | Rows after: 804
[Step 3] Missing values in 'id': 0
[Step 3] Missing values in 'name': 0
[Step 4] Rows removed due to missing critical info: 0 | Rows after: 804
[Step 5] Index reset. Final stations count: 804


,id,name
0,30200,9th St & Pennsylvania Ave NW
1,30201,9th & G St NW
2,31000,Eads St & 15th St S
3,31002,Crystal Dr & 20th St S
4,31003,Crystal Dr & 15th St S


Cleaning and Standardizing Parking Zones GeoData

In [20]:
# 1. Load the GeoJSON file as a GeoDataFrame
zones = gpd.read_file("datasets/Residential_and_Visitor_Parking_Zones.geojson")
print(f"[Step 1] Loaded zones: {len(zones)} polygons")


# 2. Check and standardize the Coordinate Reference System (CRS)
print(f"[Step 2] CRS before conversion: {zones.crs}")
if zones.crs is None or zones.crs.to_string() != "EPSG:4326":
    zones = zones.to_crs("EPSG:4326")
print(f"[Step 2] CRS after conversion: {zones.crs}")


# 3. Check for duplicate polygons by unique identifier
before = len(zones)
zones = zones.drop_duplicates(subset=['OBJECTID'])
after = len(zones)
print(f"[Step 3] Duplicates removed: {before - after} | Polygons after: {after}")



# 4. Check for null or invalid zone names
missing_names = zones['NAME'].isnull().sum()
missing_zones = zones['RPP_ZONE'].isnull().sum()
print(f"[Step 4] Zones with missing 'NAME': {missing_names}")
print(f"[Step 4] Zones with missing 'RPP_ZONE': {missing_zones}")



# 5. Remove polygons with null names or IDs
before = len(zones)
zones = zones.dropna(subset=['NAME', 'RPP_ZONE'])
after = len(zones)
print(f"[Step 5] Zones removed due to missing names or IDs: {before - after} | Polygons after: {after}")


# 6. Check geometry validity
invalid_geoms = zones[~zones.is_valid]
print(f"[Step 6] Invalid geometries: {len(invalid_geoms)}")
zones['geometry'] = zones['geometry'].buffer(0)

display(zones.head(40))



[Step 1] Loaded zones: 40 polygons
[Step 2] CRS before conversion: EPSG:4326
[Step 2] CRS after conversion: EPSG:4326
[Step 3] Duplicates removed: 0 | Polygons after: 40
[Step 4] Zones with missing 'NAME': 0
[Step 4] Zones with missing 'RPP_ZONE': 0
[Step 5] Zones removed due to missing names or IDs: 0 | Polygons after: 40
[Step 6] Invalid geometries: 0


,NAME,RPP_ZONE,ANC_ID,WEB_URL,GIS_ID,SHAPE_LENG,CREATOR,CREATED,EDITOR,EDITED,OBJECTID,SHAPEAREA,SHAPELEN,geometry
0,ANC 4A,4,4A,http://anc.dc.gov/page/advisory-neighborhood-c...,Res_Visitor_Parking_Zone_17,19149.358478,None,None,None,None,1,0,0,"POLYGON ((-77.03331 38.98999, -77.02662 38.984..."
1,ANC 3G,3,3G,http://anc.dc.gov/page/advisory-neighborhood-c...,Res_Visitor_Parking_Zone_16,13274.053932,None,None,None,None,2,0,0,"POLYGON ((-77.05161 38.98615, -77.05141 38.986..."
2,ANC 4B,4,4B,http://anc.dc.gov/page/advisory-neighborhood-c...,Res_Visitor_Parking_Zone_10,10937.574323,None,None,None,None,3,0,0,"POLYGON ((-76.99908 38.96328, -76.99143 38.957..."
3,ANC 3F,3,3F,http://anc.dc.gov/page/advisory-neighborhood-c...,Res_Visitor_Parking_Zone_15,11804.927060,None,None,None,None,4,0,0,"POLYGON ((-77.04702 38.96038, -77.04666 38.960..."
4,ANC 4C,4,4C,http://anc.dc.gov/page/advisory-neighborhood-c...,Res_Visitor_Parking_Zone_7,9774.918003,None,None,None,None,5,0,0,"POLYGON ((-77.02803 38.95622, -77.02802 38.956..."
5,ANC 4D,4,4D,http://anc.dc.gov/page/advisory-neighborhood-c...,Res_Visitor_Parking_Zone_33,6644.071961,None,None,None,None,6,0,0,"POLYGON ((-77.01923 38.9576, -77.01922 38.9575..."
6,ANC 5A,5,5A,http://anc.dc.gov/page/advisory-neighborhood-c...,Res_Visitor_Parking_Zone_34,12743.869490,None,None,None,None,7,0,0,"POLYGON ((-76.99143 38.9573, -76.98559 38.9527..."
7,ANC 5B,5,5B,http://anc.dc.gov/page/advisory-neighborhood-c...,Res_Visitor_Parking_Zone_36,11542.810028,None,None,None,None,8,0,0,"POLYGON ((-76.98239 38.95024, -76.96666 38.937..."
8,ANC 3D,3,3D,http://anc.dc.gov/page/advisory-neighborhood-c...,Res_Visitor_Parking_Zone_14,16117.574199,None,None,None,None,9,0,0,"POLYGON ((-77.1011 38.94854, -77.10103 38.9485..."
9,ANC 3C,3,3C,http://anc.dc.gov/page/advisory-neighborhood-c...,Res_Visitor_Parking_Zone_13,11933.421314,None,None,None,None,10,0,0,"POLYGON ((-77.07789 38.94317, -77.07724 38.943..."


Data Cleaning and GeoDataFrame Creation for Capital Bikeshare Station Locations

In [21]:

# 1. Load the stations data
Bikeshare = pd.read_csv("datasets/Capital_Bikeshare_Locations.csv")
print(f"[Step 1] Loaded stations: {len(Bikeshare)} rows")

# 2. Remove duplicate stations by unique identifier
id_column = 'NAME' if 'NAME' in Bikeshare.columns else Bikeshare.columns[0]
before = len(Bikeshare)
Bikeshare = Bikeshare.drop_duplicates(subset=[id_column])
after = len(Bikeshare)
print(f"[Step 2] Duplicates removed: {before - after} | Rows after: {after}")

# 3. Check for missing values in critical columns
critical_cols = ['NAME', 'LATITUDE', 'LONGITUDE']
for col in critical_cols:
    missing = Bikeshare[col].isnull().sum()
    print(f"[Step 3] Missing values in '{col}': {missing}")

# 4. Remove rows with missing station name or coordinates
before = len(Bikeshare)
Bikeshare = Bikeshare.dropna(subset=critical_cols)
after = len(Bikeshare)
print(f"[Step 4] Rows removed due to missing critical info: {before - after} | Rows after: {after}")

# 5. Show null values in all columns
print("[Step 5] Null values per column:")
print(Bikeshare.isnull().sum())

# 6. Drop the columns 'STATION_STATUS' and 'GIS_ID'
Bikeshare = Bikeshare.drop(columns=['STATION_STATUS', 'GIS_ID'])
print("[Step 6] Dropped columns: 'STATION_STATUS', 'GIS_ID'")

# 7. Fill null values in 'REGION_ID' with 'Unknown'
Bikeshare['REGION_ID'] = Bikeshare['REGION_ID'].fillna('Unknown')
print(f"[Step 7] Null values in 'REGION_ID' after fill: {Bikeshare['REGION_ID'].isnull().sum()}")

# 8. Reset index for clarity
Bikeshare = Bikeshare.reset_index(drop=True)
print(f"[Step 8] Index reset. Final stations count: {len(Bikeshare)}")

# 9. Create 'geometry' column from LATITUDE and LONGITUDE
Bikeshare['geometry'] = Bikeshare.apply(lambda row: Point(row['LONGITUDE'], row['LATITUDE']), axis=1)
print("[Step 9] Added 'geometry' column from LATITUDE and LONGITUDE.")

# 10. Drop the original LATITUDE and LONGITUDE columns
cols_to_drop = ['LATITUDE', 'LONGITUDE']
Bikeshare = Bikeshare.drop(columns=cols_to_drop)
print("[Step 10] Dropped columns:", cols_to_drop)

# 11. Convert the DataFrame to a GeoDataFrame and set CRS
Bikeshare = gpd.GeoDataFrame(Bikeshare, geometry='geometry', crs="EPSG:4326")
print("[Step 11] Converted to GeoDataFrame with CRS:", Bikeshare.crs)

# 12. Preview cleaned GeoDataFrame
display(Bikeshare.head())


[Step 1] Loaded stations: 794 rows
[Step 2] Duplicates removed: 0 | Rows after: 794
[Step 3] Missing values in 'NAME': 0
[Step 3] Missing values in 'LATITUDE': 0
[Step 3] Missing values in 'LONGITUDE': 0
[Step 4] Rows removed due to missing critical info: 0 | Rows after: 794
[Step 5] Null values per column:
X                                   0
Y                                   0
NAME                                0
STATION_TYPE                        0
STATION_ID                          0
STATION_STATUS                    794
LAST_REPORTED                       0
NUM_DOCKS_AVAILABLE                 0
NUM_DOCKS_DISABLED                  0
NUM_BIKES_AVAILABLE                 0
NUM_EBIKES_AVAILABLE                0
NUM_BIKES_DISABLED                  0
IS_INSTALLED                        0
IS_RETURNING                        0
IS_RENTING                          0
HAS_KIOSK                           0
IOS                                 0
ANDROID                             0
ELECTRI

,X,Y,NAME,STATION_TYPE,STATION_ID,LAST_REPORTED,NUM_DOCKS_AVAILABLE,NUM_DOCKS_DISABLED,NUM_BIKES_AVAILABLE,NUM_EBIKES_AVAILABLE,...,ANDROID,ELECTRIC_BIKE_SURCHARGE_WAIVER,EIGHTD_HAS_KEY_DISPENSER,CAPACITY,RENTAL_METHODS,REGION_ID,REGION_NAME,GIS_LAST_MOD_DTTM,OBJECTID,geometry
0,-8.577104e+06,4.705679e+06,Lincoln Memorial,classic,08254284-1f3f-11e7-bf6b-3863bb334450,2025/05/05 20:25:51+00,17,0,8,0,...,https://dc.lft.to/lastmile_qr_scan,NO,NO,25,"KEY,CREDITCARD",42.0,DCA-CABI,2025/05/05 20:27:29+00,261284028,POINT (-77.04944 38.88826)
1,-8.609481e+06,4.714716e+06,W&OD Trail/Sunset Hills Rd & Isaac Newton Sq,classic,08263fbd-1f3f-11e7-bf6b-3863bb334450,2025/05/05 20:25:51+00,15,0,3,0,...,https://dc.lft.to/lastmile_qr_scan,NO,NO,19,"KEY,CREDITCARD",104.0,DCA-CABI,2025/05/05 20:27:29+00,261284029,POINT (-77.34028 38.95142)
2,-8.575867e+06,4.705657e+06,17th St & Independence Ave SW,classic,082623bf-1f3f-11e7-bf6b-3863bb334450,2025/05/05 20:25:51+00,10,0,6,4,...,https://dc.lft.to/lastmile_qr_scan,NO,NO,19,"KEY,CREDITCARD",42.0,DCA-CABI,2025/05/05 20:27:29+00,261284030,POINT (-77.03832 38.8881)
3,-8.574188e+06,4.706622e+06,8th & D St NW,classic,08256ac9-1f3f-11e7-bf6b-3863bb334450,2025/05/05 20:25:49+00,0,0,24,0,...,https://dc.lft.to/lastmile_qr_scan,NO,NO,24,"KEY,CREDITCARD",42.0,DCA-CABI,2025/05/05 20:27:29+00,261284031,POINT (-77.02324 38.89485)
4,-8.567161e+06,4.706864e+06,Anacostia Ave & Benning Rd NE / River Terrace,classic,082518eb-1f3f-11e7-bf6b-3863bb334450,2025/05/05 20:25:49+00,4,0,10,0,...,https://dc.lft.to/lastmile_qr_scan,NO,NO,15,"KEY,CREDITCARD",42.0,DCA-CABI,2025/05/05 20:27:29+00,261284032,POINT (-76.96012 38.89654)


Weather Data: Loading, Preview, and Null Value Check

In [22]:
# 1. Load the weather data
weather = pd.read_csv("datasets/Washington,DC,USA 2024-01-01 to 2024-12-31.csv")
print(f"[Step 1] Loaded weather data: {len(weather)} rows")

# 2. Show columns and preview data
print(f"[Step 2] Columns: {weather.columns.tolist()}")
display(weather.head())


# 5. Show null values in all columns
print("[Step 5] Null values per column:")
print(weather.isnull().sum())

[Step 1] Loaded weather data: 366 rows
[Step 2] Columns: ['name', 'datetime', 'tempmax', 'tempmin', 'temp', 'humidity', 'windspeed', 'windspeedmax', 'windspeedmean', 'windspeedmin', 'cloudcover', 'sunrise', 'sunset', 'moonphase', 'conditions', 'description', 'icon']


,name,datetime,tempmax,tempmin,temp,humidity,windspeed,windspeedmax,windspeedmean,windspeedmin,cloudcover,sunrise,sunset,moonphase,conditions,description,icon
0,"Washington,DC,USA",2024-01-01,6.1,3.8,4.9,74.7,21.9,21.9,11.9,0.0,90.3,2024-01-01T07:26:50,2024-01-01T16:56:30,0.68,"Rain, Overcast",Cloudy skies throughout the day with early mor...,rain
1,"Washington,DC,USA",2024-01-02,7.7,2.0,4.7,57.3,29.2,29.2,18.6,6.7,47.5,2024-01-02T07:26:58,2024-01-02T16:57:20,0.72,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day
2,"Washington,DC,USA",2024-01-03,8.8,0.9,4.4,59.5,14.2,14.2,8.9,0.7,68.7,2024-01-03T07:27:04,2024-01-03T16:58:10,0.75,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day
3,"Washington,DC,USA",2024-01-04,8.1,0.7,4.7,55.2,35.0,35.0,17.0,0.0,68.5,2024-01-04T07:27:07,2024-01-04T16:59:02,0.78,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day
4,"Washington,DC,USA",2024-01-05,4.4,-2.4,0.9,55.0,21.3,21.3,10.6,5.1,44.5,2024-01-05T07:27:09,2024-01-05T16:59:55,0.81,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day


[Step 5] Null values per column:
name             0
datetime         0
tempmax          0
tempmin          0
temp             0
humidity         0
windspeed        0
windspeedmax     0
windspeedmean    0
windspeedmin     0
cloudcover       0
sunrise          0
sunset           0
moonphase        0
conditions       0
description      0
icon             0
dtype: int64


Spatial Join of Trips with Residential Zones and Merging Zone Info

In [23]:
# 1. Spatial join for start points
gdf_start_with_zone = gpd.sjoin(
    gdf_start, 
    zones[['NAME', 'GIS_ID', 'OBJECTID', 'geometry']],
    how='left',
    predicate='within'
)

# Add zone geometry (polygon) from zones using index_right
gdf_start_with_zone['Zone_start_geometry'] = gdf_start_with_zone['index_right'].map(zones['geometry'])

# Rename columns for clarity
gdf_start_with_zone = gdf_start_with_zone.rename(columns={
    'NAME': 'Zone_start_zone_name',
    'GIS_ID': 'Zone_start_GIS_ID',
    'OBJECTID': 'Zone_start_OBJECTID'
})

# Select required columns
gdf_start_with_zone = gdf_start_with_zone[
    ['ride_id', 'Zone_start_zone_name', 'Zone_start_GIS_ID', 'Zone_start_OBJECTID', 'Zone_start_geometry']
]
print("[Step 1] Added residential zone info (with geometry) for trip start points.")

# 2. Spatial join for end points
gdf_end_with_zone = gpd.sjoin(
    gdf_end, 
    zones[['NAME', 'GIS_ID', 'OBJECTID', 'geometry']],
    how='left',
    predicate='within'
)

gdf_end_with_zone['Zone_end_geometry'] = gdf_end_with_zone['index_right'].map(zones['geometry'])
gdf_end_with_zone = gdf_end_with_zone.rename(columns={
    'NAME': 'Zone_end_zone_name',
    'GIS_ID': 'Zone_end_GIS_ID',
    'OBJECTID': 'Zone_end_OBJECTID'
})

gdf_end_with_zone = gdf_end_with_zone[
    ['ride_id', 'Zone_end_zone_name', 'Zone_end_GIS_ID', 'Zone_end_OBJECTID', 'Zone_end_geometry']
]
print("[Step 2] Added residential zone info (with geometry) for trip end points.")

# 3. Merge start zone info into the main daily trips table
df_with_start_zone = df_cleaned.merge(
    gdf_start_with_zone,
    on='ride_id',
    how='left'
)
print("[Step 3] Merged start zone info with daily trips table.")

# 4. Merge end zone info into the result
df_with_start_end_zones = df_with_start_zone.merge(
    gdf_end_with_zone,
    on='ride_id',
    how='left',
    suffixes=('', '_end')
)
print("[Step 4] Merged end zone info with daily trips table.")

# 5. Show a sample of the final table with all trip and zone information
with pd.option_context('display.max_columns', None):
    display(df_with_start_end_zones.head(5))

[Step 1] Added residential zone info (with geometry) for trip start points.
[Step 2] Added residential zone info (with geometry) for trip end points.
[Step 3] Merged start zone info with daily trips table.
[Step 4] Merged end zone info with daily trips table.


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,member_casual,duration_min,geometry_start,geometry_end,Zone_start_zone_name,Zone_start_GIS_ID,Zone_start_OBJECTID,Zone_start_geometry,Zone_end_zone_name,Zone_end_GIS_ID,Zone_end_OBJECTID,Zone_end_geometry
0,6A6067934F2E1EC9,classic_bike,2024-05-04 14:27:37,2024-05-04 14:37:52,14th & G St NW,31238,Jefferson Memorial,31249,casual,10.250000,POINT (-77.03189760446548 38.89816396413188),POINT (-77.037413 38.879819),ANC 2C,Res_Visitor_Parking_Zone_5,27.0,"POLYGON ((-77.02405 38.90294, -77.02405 38.902...",ANC 2A,Res_Visitor_Parking_Zone_3,25.0,"POLYGON ((-77.05445 38.90725, -77.05317 38.907..."
1,FF5D6FE256D4DE34,classic_bike,2024-05-04 14:00:53,2024-05-04 15:00:25,Jefferson Dr & 14th St SW,31247,7th & F St NW / National Portrait Gallery,31232,casual,59.533333,POINT (-77.032427 38.8885525),POINT (-77.02219069004059 38.89728304853854),ANC 2C,Res_Visitor_Parking_Zone_5,27.0,"POLYGON ((-77.02405 38.90294, -77.02405 38.902...",ANC 2C,Res_Visitor_Parking_Zone_5,27.0,"POLYGON ((-77.02405 38.90294, -77.02405 38.902..."
2,4A64EABCDCB7506A,classic_bike,2024-05-04 15:12:21,2024-05-04 15:27:29,1st & M St NE,31603,4th & M St SW,31108,casual,15.133333,POINT (-77.005483 38.905697),POINT (-77.01777443289757 38.87682269362588),ANC 6C,Res_Visitor_Parking_Zone_21,24.0,"POLYGON ((-76.99496 38.90141, -76.99496 38.901...",ANC 6D,Res_Visitor_Parking_Zone_22,31.0,"POLYGON ((-77.01044 38.8876, -77.01013 38.8876..."
3,0F7347CF6409CEB7,classic_bike,2024-05-04 14:49:59,2024-05-04 14:54:11,Jefferson Memorial,31249,Hains Point/Buckeye & Ohio Dr SW,31273,casual,4.200000,POINT (-77.037413 38.879819),POINT (-77.03023 38.878433),ANC 2A,Res_Visitor_Parking_Zone_3,25.0,"POLYGON ((-77.05445 38.90725, -77.05317 38.907...",ANC 6D,Res_Visitor_Parking_Zone_22,31.0,"POLYGON ((-77.01044 38.8876, -77.01013 38.8876..."
4,2022D12F38B197C3,classic_bike,2024-05-04 19:23:20,2024-05-04 19:41:35,New Hampshire Ave & T St NW,31229,14th & G St NW,31238,casual,18.250000,POINT (-77.038252 38.915544),POINT (-77.03189760446548 38.89816396413188),ANC 2B,Res_Visitor_Parking_Zone_4,20.0,"POLYGON ((-77.04123 38.91702, -77.04114 38.917...",ANC 2C,Res_Visitor_Parking_Zone_5,27.0,"POLYGON ((-77.02405 38.90294, -77.02405 38.902..."


Merging Daily Trip Data with Weather Information

In [24]:
# 1. Load the final trips table
trips = df_with_start_end_zones

# 2. Load weather data and keep only required columns
weather = pd.read_csv("datasets/Washington,DC,USA 2024-01-01 to 2024-12-31.csv")
weather = weather[['datetime', 'temp', 'windspeedmean', 'sunrise', 'sunset', 'conditions']]
print("[Step 2] Loaded weather data and selected required columns.")

# 3. Rename weather columns by adding 'Weather' prefix
weather = weather.rename(columns={
    'temp': 'Weather_temp',
    'windspeedmean': 'Weather_windspeedmean',
    'sunrise': 'Weather_sunrise',
    'sunset': 'Weather_sunset',
    'conditions': 'Weather_conditions'
})

# 4. Make sure date columns are datetime type
trips['started_at'] = pd.to_datetime(trips['started_at'])
weather['datetime'] = pd.to_datetime(weather['datetime'])

# 5. Create a 'date' column in trips table for merging
trips['date'] = trips['started_at'].dt.date
weather['date'] = weather['datetime'].dt.date

# 6. Merge trips with weather on date
trips_with_weather = trips.merge(
    weather[['date', 'Weather_temp', 'Weather_windspeedmean', 'Weather_sunrise', 'Weather_sunset', 'Weather_conditions']], 
    on='date', 
    how='left'
)
print("[Step 6] Merged trips with weather data (with Weather prefix).")

# 7. Show a sample of the result
with pd.option_context('display.max_columns', None):
    display(trips_with_weather.head())

[Step 2] Loaded weather data and selected required columns.
[Step 6] Merged trips with weather data (with Weather prefix).


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,member_casual,duration_min,geometry_start,geometry_end,Zone_start_zone_name,Zone_start_GIS_ID,Zone_start_OBJECTID,Zone_start_geometry,Zone_end_zone_name,Zone_end_GIS_ID,Zone_end_OBJECTID,Zone_end_geometry,date,Weather_temp,Weather_windspeedmean,Weather_sunrise,Weather_sunset,Weather_conditions
0,6A6067934F2E1EC9,classic_bike,2024-05-04 14:27:37,2024-05-04 14:37:52,14th & G St NW,31238,Jefferson Memorial,31249,casual,10.250000,POINT (-77.03189760446548 38.89816396413188),POINT (-77.037413 38.879819),ANC 2C,Res_Visitor_Parking_Zone_5,27.0,"POLYGON ((-77.02405 38.90294, -77.02405 38.902...",ANC 2A,Res_Visitor_Parking_Zone_3,25.0,"POLYGON ((-77.05445 38.90725, -77.05317 38.907...",2024-05-04,12.3,12.5,2024-05-04T06:06:12,2024-05-04T20:04:13,"Rain, Overcast"
1,FF5D6FE256D4DE34,classic_bike,2024-05-04 14:00:53,2024-05-04 15:00:25,Jefferson Dr & 14th St SW,31247,7th & F St NW / National Portrait Gallery,31232,casual,59.533333,POINT (-77.032427 38.8885525),POINT (-77.02219069004059 38.89728304853854),ANC 2C,Res_Visitor_Parking_Zone_5,27.0,"POLYGON ((-77.02405 38.90294, -77.02405 38.902...",ANC 2C,Res_Visitor_Parking_Zone_5,27.0,"POLYGON ((-77.02405 38.90294, -77.02405 38.902...",2024-05-04,12.3,12.5,2024-05-04T06:06:12,2024-05-04T20:04:13,"Rain, Overcast"
2,4A64EABCDCB7506A,classic_bike,2024-05-04 15:12:21,2024-05-04 15:27:29,1st & M St NE,31603,4th & M St SW,31108,casual,15.133333,POINT (-77.005483 38.905697),POINT (-77.01777443289757 38.87682269362588),ANC 6C,Res_Visitor_Parking_Zone_21,24.0,"POLYGON ((-76.99496 38.90141, -76.99496 38.901...",ANC 6D,Res_Visitor_Parking_Zone_22,31.0,"POLYGON ((-77.01044 38.8876, -77.01013 38.8876...",2024-05-04,12.3,12.5,2024-05-04T06:06:12,2024-05-04T20:04:13,"Rain, Overcast"
3,0F7347CF6409CEB7,classic_bike,2024-05-04 14:49:59,2024-05-04 14:54:11,Jefferson Memorial,31249,Hains Point/Buckeye & Ohio Dr SW,31273,casual,4.200000,POINT (-77.037413 38.879819),POINT (-77.03023 38.878433),ANC 2A,Res_Visitor_Parking_Zone_3,25.0,"POLYGON ((-77.05445 38.90725, -77.05317 38.907...",ANC 6D,Res_Visitor_Parking_Zone_22,31.0,"POLYGON ((-77.01044 38.8876, -77.01013 38.8876...",2024-05-04,12.3,12.5,2024-05-04T06:06:12,2024-05-04T20:04:13,"Rain, Overcast"
4,2022D12F38B197C3,classic_bike,2024-05-04 19:23:20,2024-05-04 19:41:35,New Hampshire Ave & T St NW,31229,14th & G St NW,31238,casual,18.250000,POINT (-77.038252 38.915544),POINT (-77.03189760446548 38.89816396413188),ANC 2B,Res_Visitor_Parking_Zone_4,20.0,"POLYGON ((-77.04123 38.91702, -77.04114 38.917...",ANC 2C,Res_Visitor_Parking_Zone_5,27.0,"POLYGON ((-77.02405 38.90294, -77.02405 38.902...",2024-05-04,12.3,12.5,2024-05-04T06:06:12,2024-05-04T20:04:13,"Rain, Overcast"


Merging Trip Records with Detailed Bikeshare Station Attributes

In [86]:
# 1. Load the main trips table
trips = trips_with_weather

# 2. Load the stations data and keep only required columns
station_cols = ['NAME', 'STATION_TYPE', 'CAPACITY', 'REGION_ID', 'REGION_NAME', 'geometry']
BikeshareNew = Bikeshare[station_cols]

# 3. Rename columns by adding "Bikeshare_" prefix
BikeshareNew = BikeshareNew.rename(columns={col: f"Bikeshare_{col}" for col in station_cols})

# 4. Merge for start station info
trips = trips.merge(
    BikeshareNew, 
    left_on='start_station_name', right_on='Bikeshare_NAME', 
    how='left', 
    suffixes=('', '_start')
)

# 5. Rename columns for start station to add "_start" suffix
for col in station_cols:
    trips = trips.rename(columns={f"Bikeshare_{col}": f"Bikeshare_{col}_start"})

# 6. Merge for end station info (you'll need to reload/reuse the Bikeshare DataFrame)
BikeshareNew = Bikeshare[station_cols]
BikeshareNew = BikeshareNew.rename(columns={col: f"Bikeshare_{col}" for col in station_cols})

trips = trips.merge(
    BikeshareNew, 
    left_on='end_station_name', right_on='Bikeshare_NAME', 
    how='left', 
    suffixes=('', '_end')
)

# 7. Rename columns for end station to add "_end" suffix
for col in station_cols:
    trips = trips.rename(columns={f"Bikeshare_{col}": f"Bikeshare_{col}_end"})

# 8. display data
with pd.option_context('display.max_columns', None):
    display(trips.head(20))

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,member_casual,duration_min,geometry_start,geometry_end,Zone_start_zone_name,Zone_start_GIS_ID,Zone_start_OBJECTID,Zone_start_geometry,Zone_end_zone_name,Zone_end_GIS_ID,Zone_end_OBJECTID,Zone_end_geometry,date,Weather_temp,Weather_windspeedmean,Weather_sunrise,Weather_sunset,Weather_conditions,Bikeshare_NAME_start,Bikeshare_STATION_TYPE_start,Bikeshare_CAPACITY_start,Bikeshare_REGION_ID_start,Bikeshare_REGION_NAME_start,Bikeshare_geometry_start,Bikeshare_NAME_end,Bikeshare_STATION_TYPE_end,Bikeshare_CAPACITY_end,Bikeshare_REGION_ID_end,Bikeshare_REGION_NAME_end,Bikeshare_geometry_end
0,6A6067934F2E1EC9,classic_bike,2024-05-04 14:27:37,2024-05-04 14:37:52,14th & G St NW,31238,Jefferson Memorial,31249,casual,10.250000,POINT (-77.03189760446548 38.89816396413188),POINT (-77.037413 38.879819),ANC 2C,Res_Visitor_Parking_Zone_5,27.0,"POLYGON ((-77.02405 38.90294, -77.02405 38.902...",ANC 2A,Res_Visitor_Parking_Zone_3,25.0,"POLYGON ((-77.05445 38.90725, -77.05317 38.907...",2024-05-04,12.3,12.5,2024-05-04T06:06:12,2024-05-04T20:04:13,"Rain, Overcast",14th & G St NW,classic,23.0,42.0,DCA-CABI,POINT (-77.0319 38.89816),Jefferson Memorial,classic,23.0,42.0,DCA-CABI,POINT (-77.03741 38.87982)
1,FF5D6FE256D4DE34,classic_bike,2024-05-04 14:00:53,2024-05-04 15:00:25,Jefferson Dr & 14th St SW,31247,7th & F St NW / National Portrait Gallery,31232,casual,59.533333,POINT (-77.032427 38.8885525),POINT (-77.02219069004059 38.89728304853854),ANC 2C,Res_Visitor_Parking_Zone_5,27.0,"POLYGON ((-77.02405 38.90294, -77.02405 38.902...",ANC 2C,Res_Visitor_Parking_Zone_5,27.0,"POLYGON ((-77.02405 38.90294, -77.02405 38.902...",2024-05-04,12.3,12.5,2024-05-04T06:06:12,2024-05-04T20:04:13,"Rain, Overcast",Jefferson Dr & 14th St SW,classic,23.0,42.0,DCA-CABI,POINT (-77.03243 38.88855),7th & F St NW / National Portrait Gallery,classic,35.0,42.0,DCA-CABI,POINT (-77.02222 38.89732)
2,4A64EABCDCB7506A,classic_bike,2024-05-04 15:12:21,2024-05-04 15:27:29,1st & M St NE,31603,4th & M St SW,31108,casual,15.133333,POINT (-77.005483 38.905697),POINT (-77.01777443289757 38.87682269362588),ANC 6C,Res_Visitor_Parking_Zone_21,24.0,"POLYGON ((-76.99496 38.90141, -76.99496 38.901...",ANC 6D,Res_Visitor_Parking_Zone_22,31.0,"POLYGON ((-77.01044 38.8876, -77.01013 38.8876...",2024-05-04,12.3,12.5,2024-05-04T06:06:12,2024-05-04T20:04:13,"Rain, Overcast",1st & M St NE,classic,15.0,42.0,DCA-CABI,POINT (-77.00548 38.9057),4th & M St SW,classic,22.0,42.0,DCA-CABI,POINT (-77.01781 38.87694)
3,0F7347CF6409CEB7,classic_bike,2024-05-04 14:49:59,2024-05-04 14:54:11,Jefferson Memorial,31249,Hains Point/Buckeye & Ohio Dr SW,31273,casual,4.200000,POINT (-77.037413 38.879819),POINT (-77.03023 38.878433),ANC 2A,Res_Visitor_Parking_Zone_3,25.0,"POLYGON ((-77.05445 38.90725, -77.05317 38.907...",ANC 6D,Res_Visitor_Parking_Zone_22,31.0,"POLYGON ((-77.01044 38.8876, -77.01013 38.8876...",2024-05-04,12.3,12.5,2024-05-04T06:06:12,2024-05-04T20:04:13,"Rain, Overcast",Jefferson Memorial,classic,23.0,42.0,DCA-CABI,POINT (-77.03741 38.87982),Hains Point/Buckeye & Ohio Dr SW,classic,17.0,42.0,DCA-CABI,POINT (-77.03023 38.87843)
4,2022D12F38B197C3,classic_bike,2024-05-04 19:23:20,2024-05-04 19:41:35,New Hampshire Ave & T St NW,31229,14th & G St NW,31238,casual,18.250000,POINT (-77.038252 38.915544),POINT (-77.03189760446548 38.89816396413188),ANC 2B,Res_Visitor_Parking_Zone_4,20.0,"POLYGON ((-77.04123 38.91702, -77.04114 38.917...",ANC 2C,Res_Visitor_Parking_Zone_5,27.0,"POLYGON ((-77.02405 38.90294, -77.02405 38.902...",2024-05-04,12.3,12.5,2024-05-04T06:06:12,2024-05-04T20:04:13,"Rain, Overcast",New Hampshire Ave & T St NW,classic,23.0,42.0,DCA-CABI,POINT (-77.03825 38.91554),14th & G St NW,classic,23.0,42.0,DCA-CABI,POINT (-77.0319 38.89816)
5,7DF015F77985083F,classic_bike,2024-05-04 17:44:03,2024-05-04 17:52:50,19th St & Constitution Ave NW,31235,17th & G St NW,31277,casual,8.783333,POINT

Merging Detailed Start and End Station Data with Trips Table

In [87]:
# 1. Prepare start and end station tables with proper prefixes
start_station = stations.rename(
    columns={'id': 'start_station_id', 'name': 'StartStation_NAME'}
)
end_station = stations.rename(
    columns={'id': 'end_station_id', 'name': 'EndStation_NAME'}
)

# 2. Load the merged trips table
trips_merged = trips
print(f"[Step 1] Loaded merged trips: {len(trips_merged)} rows")

# 3. Merge start station info
trips_with_start_station = trips_merged.merge(
    start_station, on='start_station_id', how='left'
)
print("[Step 2] Merged start station info.")

# 4. Merge end station info
trips_with_stations = trips_with_start_station.merge(
    end_station, on='end_station_id', how='left'
)
print("[Step 3] Merged end station info.")



# 6. Show a sample of the final merged table
with pd.option_context('display.max_columns', None):
    display(trips_with_stations.head())

[Step 1] Loaded merged trips: 1980232 rows
[Step 2] Merged start station info.
[Step 3] Merged end station info.


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,member_casual,duration_min,geometry_start,geometry_end,Zone_start_zone_name,Zone_start_GIS_ID,Zone_start_OBJECTID,Zone_start_geometry,Zone_end_zone_name,Zone_end_GIS_ID,Zone_end_OBJECTID,Zone_end_geometry,date,Weather_temp,Weather_windspeedmean,Weather_sunrise,Weather_sunset,Weather_conditions,Bikeshare_NAME_start,Bikeshare_STATION_TYPE_start,Bikeshare_CAPACITY_start,Bikeshare_REGION_ID_start,Bikeshare_REGION_NAME_start,Bikeshare_geometry_start,Bikeshare_NAME_end,Bikeshare_STATION_TYPE_end,Bikeshare_CAPACITY_end,Bikeshare_REGION_ID_end,Bikeshare_REGION_NAME_end,Bikeshare_geometry_end,StartStation_NAME,EndStation_NAME
0,6A6067934F2E1EC9,classic_bike,2024-05-04 14:27:37,2024-05-04 14:37:52,14th & G St NW,31238,Jefferson Memorial,31249,casual,10.250000,POINT (-77.03189760446548 38.89816396413188),POINT (-77.037413 38.879819),ANC 2C,Res_Visitor_Parking_Zone_5,27.0,"POLYGON ((-77.02405 38.90294, -77.02405 38.902...",ANC 2A,Res_Visitor_Parking_Zone_3,25.0,"POLYGON ((-77.05445 38.90725, -77.05317 38.907...",2024-05-04,12.3,12.5,2024-05-04T06:06:12,2024-05-04T20:04:13,"Rain, Overcast",14th & G St NW,classic,23.0,42.0,DCA-CABI,POINT (-77.0319 38.89816),Jefferson Memorial,classic,23.0,42.0,DCA-CABI,POINT (-77.03741 38.87982),14th & G St NW,Jefferson Memorial
1,FF5D6FE256D4DE34,classic_bike,2024-05-04 14:00:53,2024-05-04 15:00:25,Jefferson Dr & 14th St SW,31247,7th & F St NW / National Portrait Gallery,31232,casual,59.533333,POINT (-77.032427 38.8885525),POINT (-77.02219069004059 38.89728304853854),ANC 2C,Res_Visitor_Parking_Zone_5,27.0,"POLYGON ((-77.02405 38.90294, -77.02405 38.902...",ANC 2C,Res_Visitor_Parking_Zone_5,27.0,"POLYGON ((-77.02405 38.90294, -77.02405 38.902...",2024-05-04,12.3,12.5,2024-05-04T06:06:12,2024-05-04T20:04:13,"Rain, Overcast",Jefferson Dr & 14th St SW,classic,23.0,42.0,DCA-CABI,POINT (-77.03243 38.88855),7th & F St NW / National Portrait Gallery,classic,35.0,42.0,DCA-CABI,POINT (-77.02222 38.89732),Jefferson Dr & 14th St SW,7th & F St NW / National Portrait Gallery
2,4A64EABCDCB7506A,classic_bike,2024-05-04 15:12:21,2024-05-04 15:27:29,1st & M St NE,31603,4th & M St SW,31108,casual,15.133333,POINT (-77.005483 38.905697),POINT (-77.01777443289757 38.87682269362588),ANC 6C,Res_Visitor_Parking_Zone_21,24.0,"POLYGON ((-76.99496 38.90141, -76.99496 38.901...",ANC 6D,Res_Visitor_Parking_Zone_22,31.0,"POLYGON ((-77.01044 38.8876, -77.01013 38.8876...",2024-05-04,12.3,12.5,2024-05-04T06:06:12,2024-05-04T20:04:13,"Rain, Overcast",1st & M St NE,classic,15.0,42.0,DCA-CABI,POINT (-77.00548 38.9057),4th & M St SW,classic,22.0,42.0,DCA-CABI,POINT (-77.01781 38.87694),1st & M St NE,4th & M St SW
3,0F7347CF6409CEB7,classic_bike,2024-05-04 14:49:59,2024-05-04 14:54:11,Jefferson Memorial,31249,Hains Point/Buckeye & Ohio Dr SW,31273,casual,4.200000,POINT (-77.037413 38.879819),POINT (-77.03023 38.878433),ANC 2A,Res_Visitor_Parking_Zone_3,25.0,"POLYGON ((-77.05445 38.90725, -77.05317 38.907...",ANC 6D,Res_Visitor_Parking_Zone_22,31.0,"POLYGON ((-77.01044 38.8876, -77.01013 38.8876...",2024-05-04,12.3,12.5,2024-05-04T06:06:12,2024-05-04T20:04:13,"Rain, Overcast",Jefferson Memorial,classic,23.0,42.0,DCA-CABI,POINT (-77.03741 38.87982),Hains Point/Buckeye & Ohio Dr SW,classic,17.0,42.0,DCA-CABI,POINT (-77.03023 38.87843),Jefferson Memorial,Hains Point/Buckeye & Ohio Dr SW
4,2022D12F38B197C3,classic_bike,2024-05-04 19:23:20,2024-05-04 19:41:35,New Hampshire Ave & T St NW,31229,14th & G St NW,31238,casual,18.250000,POINT (-77.038252 38.915544),POINT (-77.03189760446548 38.89816396413188),ANC 2B,Res_Visitor_Parking_Zone_4,20.0,"POLYGON ((-77.04123 38.91702, -77.04114 38.917...",ANC 2C,Res_Visitor_Parking_Zone_5,27.0,"POLYGON ((-77.02405 38.90294, -77.02405 38.902...",2024-05-04,12.3,12.5,2024-05-04T06:06:12,2024-05-04T20:04:13,"Rain, Overcast",New Hampshire Ave & T St NW,classic,23.0,42.0,DCA-CABI,POINT (-77.03825 38.91554),14

In [88]:
print(trips_with_stations.info(memory_usage='deep'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1980232 entries, 0 to 1980231
Data columns (total 40 columns):
 #   Column                        Dtype         
---  ------                        -----         
 0   ride_id                       object        
 1   rideable_type                 object        
 2   started_at                    datetime64[ns]
 3   ended_at                      datetime64[ns]
 4   start_station_name            object        
 5   start_station_id              Int64         
 6   end_station_name              object        
 7   end_station_id                Int64         
 8   member_casual                 object        
 9   duration_min                  float64       
 10  geometry_start                object        
 11  geometry_end                  object        
 12  Zone_start_zone_name          object        
 13  Zone_start_GIS_ID             object        
 14  Zone_start_OBJECTID           float64       
 15  Zone_start_geometry           ge

In [89]:
stations = stations.drop_duplicates(subset='id')

In [90]:
# Only extract if value is a valid Point
trips_with_stations['start_longitude'] = trips_with_stations['Bikeshare_geometry_start'].apply(
    lambda geom: geom.x if isinstance(geom, Point) else None
)

trips_with_stations['start_latitude'] = trips_with_stations['Bikeshare_geometry_start'].apply(
    lambda geom: geom.y if isinstance(geom, Point) else None
)

trips_with_stations['end_longitude'] = trips_with_stations['Bikeshare_geometry_end'].apply(
    lambda geom: geom.x if isinstance(geom, Point) else None
)

trips_with_stations['end_latitude'] = trips_with_stations['Bikeshare_geometry_end'].apply(
    lambda geom: geom.y if isinstance(geom, Point) else None
)


In [91]:
trips_with_stations.drop(columns=[
    'Zone_start_geometry',
    'Zone_end_geometry',
    'Bikeshare_geometry_end',
    'Bikeshare_geometry_start',
], inplace=True)

In [113]:
trips_with_stations['rounded_start_lat'] = trips_with_stations['start_latitude'].round(5)
trips_with_stations['rounded_start_lon'] = trips_with_stations['start_longitude'].round(5)

ref_from_start = trips_with_stations[
    trips_with_stations['end_station_name'].notna() &
    trips_with_stations['start_latitude'].notna() &
    trips_with_stations['start_longitude'].notna()
    ][
    ['rounded_start_lat', 'rounded_start_lon', 
     'end_station_name', 'end_station_id',
     'end_latitude', 'end_longitude',
     'Bikeshare_NAME_end', 'Bikeshare_STATION_TYPE_end',
     'Bikeshare_CAPACITY_end', 'Bikeshare_REGION_ID_end',
     'Bikeshare_REGION_NAME_end','Zone_end_zone_name', 'Zone_end_GIS_ID',
     'Zone_end_OBJECTID', 'EndStation_NAME']
].drop_duplicates(subset=['rounded_start_lat', 'rounded_start_lon'])

ref_from_start.set_index(['rounded_start_lat', 'rounded_start_lon'], inplace=True)

end_cols_to_fill = [
    'end_station_name', 'end_station_id',
    'end_latitude', 'end_longitude',
    'Bikeshare_NAME_end', 'Bikeshare_STATION_TYPE_end',
    'Bikeshare_CAPACITY_end', 'Bikeshare_REGION_ID_end',
    'Bikeshare_REGION_NAME_end',
    'Zone_end_zone_name', 'Zone_end_GIS_ID',
    'Zone_end_OBJECTID', 'EndStation_NAME'
]

for col in end_cols_to_fill:
    trips_with_stations[col] = trips_with_stations.apply(lambda row: hf.fill_from_start_coords(row, col, ref_from_start), axis=1)

trips_with_stations.drop(columns=['rounded_start_lat', 'rounded_start_lon'], inplace=True)

In [114]:
trips_with_stations.dtypes

ride_id                                 object
rideable_type                           object
started_at                      datetime64[ns]
ended_at                        datetime64[ns]
start_station_name                      object
start_station_id                         Int64
end_station_name                        object
end_station_id                           int64
member_casual                           object
duration_min                           float64
geometry_start                          object
geometry_end                            object
Zone_start_zone_name                    object
Zone_start_GIS_ID                       object
Zone_start_OBJECTID                    float64
Zone_end_zone_name                      object
Zone_end_GIS_ID                         object
Zone_end_OBJECTID                      float64
date                                    object
Weather_temp                           float64
Weather_windspeedmean                  float64
Weather_sunri

In [115]:
trips_with_stations.isna().sum()

ride_id                              0
rideable_type                        0
started_at                           0
ended_at                             0
start_station_name                   0
start_station_id                     0
end_station_name                     0
end_station_id                       0
member_casual                        0
duration_min                         0
geometry_start                       0
geometry_end                         0
Zone_start_zone_name            266248
Zone_start_GIS_ID               266248
Zone_start_OBJECTID             266248
Zone_end_zone_name              218751
Zone_end_GIS_ID                 218751
Zone_end_OBJECTID               218751
date                                 0
Weather_temp                         4
Weather_windspeedmean                4
Weather_sunrise                      4
Weather_sunset                       4
Weather_conditions                   4
Bikeshare_NAME_start            410848
Bikeshare_STATION_TYPE_st

In [116]:
trips_with_stations['Bikeshare_REGION_ID_start'] = trips_with_stations['Bikeshare_REGION_ID_start'].astype(str)

trips_with_stations['Bikeshare_REGION_ID_end'] = trips_with_stations['Bikeshare_REGION_ID_end'].astype(str)

trips_with_stations['geometry_start'] = trips_with_stations['geometry_start'].apply(lambda geom: geom.wkt if geom else None)
trips_with_stations['geometry_end'] = trips_with_stations['geometry_end'].apply(lambda geom: geom.wkt if geom else None)

In [117]:
trips_with_stations.to_parquet('datasets/modified.parquet', index=False)

In [118]:
from shapely import wkt

trips_with_stations['geometry_start'] = trips_with_stations['geometry_start'].apply(wkt.loads)
trips_with_stations['geometry_end'] = trips_with_stations['geometry_end'].apply(wkt.loads)

In [119]:
shopping_centers = np.array([
    [38.9009, -77.0260],   # CityCenterDC
    [38.8971, -77.0064],   # Union Station
    [38.9057, -77.0631],   # Georgetown Park
    [38.8631, -77.0599],   # Pentagon City
    [38.8765, -77.0316],   # The Wharf
    [38.8741, -77.0028],   # Capitol Riverfront
    [38.9613, -77.0840],   # Friendship Heights
])

In [120]:
proximity = 300
trips_with_stations['start_near_any_mall'] = trips_with_stations.apply(
    lambda row: int(hf.distance_to_closest_mall(row['start_latitude'], row['start_longitude'], shopping_centers) <= proximity),
    axis=1
)

trips_with_stations['end_near_any_mall'] = trips_with_stations.apply(
    lambda row: int(hf.distance_to_closest_mall(row['end_latitude'], row['end_longitude'], shopping_centers) <= proximity),
    axis=1
)

In [121]:
count_of_ones = trips_with_stations['start_near_any_mall'].eq(1).sum()

print(f"Number of 1s in '{'start_near_any_mall'}': {count_of_ones}")

count_of_ones = trips_with_stations['end_near_any_mall'].eq(1).sum()

print(f"Number of 1s in '{'end_near_any_mall'}': {count_of_ones}")

Number of 1s in 'start_near_any_mall': 59765
Number of 1s in 'end_near_any_mall': 70529


b.1

Extracting Date Features from Merged Trip Data

In [122]:
# 1. Load the merged daily trips data
merged_data = trips_with_stations
print(f"[Step 1] Loaded merged trips data: {len(merged_data)} rows")

# 2. Ensure 'started_at' is datetime type
merged_data['started_at'] = pd.to_datetime(merged_data['started_at'])
print("[Step 2] Converted 'started_at' to datetime")

# 3. Extract required date features only
merged_data['year'] = merged_data['started_at'].dt.year             
merged_data['month'] = merged_data['started_at'].dt.month          
merged_data['day'] = merged_data['started_at'].dt.day          
merged_data['weekday_name'] = merged_data['started_at'].dt.day_name()

print("[Step 3] Extracted features: year, month, day, weekday_name")

# 4. Preview the result
with pd.option_context('display.max_columns', None):
    display(merged_data.head())


[Step 1] Loaded merged trips data: 1980232 rows
[Step 2] Converted 'started_at' to datetime
[Step 3] Extracted features: year, month, day, weekday_name


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,member_casual,duration_min,geometry_start,geometry_end,Zone_start_zone_name,Zone_start_GIS_ID,Zone_start_OBJECTID,Zone_end_zone_name,Zone_end_GIS_ID,Zone_end_OBJECTID,date,Weather_temp,Weather_windspeedmean,Weather_sunrise,Weather_sunset,Weather_conditions,Bikeshare_NAME_start,Bikeshare_STATION_TYPE_start,Bikeshare_CAPACITY_start,Bikeshare_REGION_ID_start,Bikeshare_REGION_NAME_start,Bikeshare_NAME_end,Bikeshare_STATION_TYPE_end,Bikeshare_CAPACITY_end,Bikeshare_REGION_ID_end,Bikeshare_REGION_NAME_end,StartStation_NAME,EndStation_NAME,start_longitude,start_latitude,end_longitude,end_latitude,start_near_any_mall,end_near_any_mall,year,month,day,weekday_name,trip_price
0,6A6067934F2E1EC9,classic_bike,2024-05-04 14:27:37,2024-05-04 14:37:52,14th & G St NW,31238,Jefferson Memorial,31249,casual,10.250000,POINT (-77.03189760446548 38.89816396413188),POINT (-77.037413 38.879819),ANC 2C,Res_Visitor_Parking_Zone_5,27.0,ANC 2A,Res_Visitor_Parking_Zone_3,25.0,2024-05-04,12.3,12.5,2024-05-04T06:06:12,2024-05-04T20:04:13,"Rain, Overcast",14th & G St NW,classic,23.0,42.0,DCA-CABI,Jefferson Memorial,classic,23.0,42.0,DCA-CABI,14th & G St NW,Jefferson Memorial,-77.031898,38.898164,-77.037413,38.879819,0,0,2024,5,4,Saturday,1.00
1,FF5D6FE256D4DE34,classic_bike,2024-05-04 14:00:53,2024-05-04 15:00:25,Jefferson Dr & 14th St SW,31247,7th & F St NW / National Portrait Gallery,31232,casual,59.533333,POINT (-77.032427 38.8885525),POINT (-77.02219069004059 38.89728304853854),ANC 2C,Res_Visitor_Parking_Zone_5,27.0,ANC 2C,Res_Visitor_Parking_Zone_5,27.0,2024-05-04,12.3,12.5,2024-05-04T06:06:12,2024-05-04T20:04:13,"Rain, Overcast",Jefferson Dr & 14th St SW,classic,23.0,42.0,DCA-CABI,7th & F St NW / National Portrait Gallery,classic,35.0,42.0,DCA-CABI,Jefferson Dr & 14th St SW,7th & F St NW / National Portrait Gallery,-77.032427,38.888553,-77.022216,38.897324,0,0,2024,5,4,Saturday,1.73
2,4A64EABCDCB7506A,classic_bike,2024-05-04 15:12:21,2024-05-04 15:27:29,1st & M St NE,31603,4th & M St SW,31108,casual,15.133333,POINT (-77.005483 38.905697),POINT (-77.01777443289757 38.87682269362588),ANC 6C,Res_Visitor_Parking_Zone_21,24.0,ANC 6D,Res_Visitor_Parking_Zone_22,31.0,2024-05-04,12.3,12.5,2024-05-04T06:06:12,2024-05-04T20:04:13,"Rain, Overcast",1st & M St NE,classic,15.0,42.0,DCA-CABI,4th & M St SW,classic,22.0,42.0,DCA-CABI,1st & M St NE,4th & M St SW,-77.005483,38.905697,-77.017807,38.876941,0,0,2024,5,4,Saturday,1.00
3,0F7347CF6409CEB7,classic_bike,2024-05-04 14:49:59,2024-05-04 14:54:11,Jefferson Memorial,31249,Hains Point/Buckeye & Ohio Dr SW,31273,casual,4.200000,POINT (-77.037413 38.879819),POINT (-77.03023 38.878433),ANC 2A,Res_Visitor_Parking_Zone_3,25.0,ANC 6D,Res_Visitor_Parking_Zone_22,31.0,2024-05-04,12.3,12.5,2024-05-04T06:06:12,2024-05-04T20:04:13,"Rain, Overcast",Jefferson Memorial,classic,23.0,42.0,DCA-CABI,Hains Point/Buckeye & Ohio Dr SW,classic,17.0,42.0,DCA-CABI,Jefferson Memorial,Hains Point/Buckeye & Ohio Dr SW,-77.037413,38.879819,-77.030230,38.878433,0,1,2024,5,4,Saturday,1.50
4,2022D12F38B197C3,classic_bike,2024-05-04 19:23:20,2024-05-04 19:41:35,New Hampshire Ave & T St NW,31229,14th & G St NW,31238,casual,18.250000,POINT (-77.038252 38.915544),POINT (-77.03189760446548 38.89816396413188),ANC 2B,Res_Visitor_Parking_Zone_4,20.0,ANC 2C,Res_Visitor_Parking_Zone_5,27.0,2024-05-04,12.3,12.5,2024-05-04T06:06:12,2024-05-04T20:04:13,"Rain, Overcast",New Hampshire Ave & T St NW,classic,23.0,42.0,DCA-CABI,14th & G St NW,classic,23.0,42.0,DCA-CABI,New Hampshire Ave & T St NW,14th & G St NW,-77.038252,38.915544,-77.031898,38.898164,0,0,2024,5,4,Saturday,1.00


b.2

Calculate Trip Duration in Minutes

It was already calculated during the data cleaning process

b.3

Custom Trip Cost Calculation Based on Fare Rules

In [123]:
merged_data['trip_price'] = merged_data.apply(hf.calculate_price, axis=1)

cbd_gdf = gpd.read_file("datasets/DDOT_Central_Business_District.geojson")
# cbd_gdf.head(10)
# merged_data['total_cost'] = merged_data.apply(calculate_cost, axis=1)
with pd.option_context('display.max_columns', None):
    display(merged_data.head())
merged_data.columns


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,member_casual,duration_min,geometry_start,geometry_end,Zone_start_zone_name,Zone_start_GIS_ID,Zone_start_OBJECTID,Zone_end_zone_name,Zone_end_GIS_ID,Zone_end_OBJECTID,date,Weather_temp,Weather_windspeedmean,Weather_sunrise,Weather_sunset,Weather_conditions,Bikeshare_NAME_start,Bikeshare_STATION_TYPE_start,Bikeshare_CAPACITY_start,Bikeshare_REGION_ID_start,Bikeshare_REGION_NAME_start,Bikeshare_NAME_end,Bikeshare_STATION_TYPE_end,Bikeshare_CAPACITY_end,Bikeshare_REGION_ID_end,Bikeshare_REGION_NAME_end,StartStation_NAME,EndStation_NAME,start_longitude,start_latitude,end_longitude,end_latitude,start_near_any_mall,end_near_any_mall,year,month,day,weekday_name,trip_price
0,6A6067934F2E1EC9,classic_bike,2024-05-04 14:27:37,2024-05-04 14:37:52,14th & G St NW,31238,Jefferson Memorial,31249,casual,10.250000,POINT (-77.03189760446548 38.89816396413188),POINT (-77.037413 38.879819),ANC 2C,Res_Visitor_Parking_Zone_5,27.0,ANC 2A,Res_Visitor_Parking_Zone_3,25.0,2024-05-04,12.3,12.5,2024-05-04T06:06:12,2024-05-04T20:04:13,"Rain, Overcast",14th & G St NW,classic,23.0,42.0,DCA-CABI,Jefferson Memorial,classic,23.0,42.0,DCA-CABI,14th & G St NW,Jefferson Memorial,-77.031898,38.898164,-77.037413,38.879819,0,0,2024,5,4,Saturday,1.00
1,FF5D6FE256D4DE34,classic_bike,2024-05-04 14:00:53,2024-05-04 15:00:25,Jefferson Dr & 14th St SW,31247,7th & F St NW / National Portrait Gallery,31232,casual,59.533333,POINT (-77.032427 38.8885525),POINT (-77.02219069004059 38.89728304853854),ANC 2C,Res_Visitor_Parking_Zone_5,27.0,ANC 2C,Res_Visitor_Parking_Zone_5,27.0,2024-05-04,12.3,12.5,2024-05-04T06:06:12,2024-05-04T20:04:13,"Rain, Overcast",Jefferson Dr & 14th St SW,classic,23.0,42.0,DCA-CABI,7th & F St NW / National Portrait Gallery,classic,35.0,42.0,DCA-CABI,Jefferson Dr & 14th St SW,7th & F St NW / National Portrait Gallery,-77.032427,38.888553,-77.022216,38.897324,0,0,2024,5,4,Saturday,1.73
2,4A64EABCDCB7506A,classic_bike,2024-05-04 15:12:21,2024-05-04 15:27:29,1st & M St NE,31603,4th & M St SW,31108,casual,15.133333,POINT (-77.005483 38.905697),POINT (-77.01777443289757 38.87682269362588),ANC 6C,Res_Visitor_Parking_Zone_21,24.0,ANC 6D,Res_Visitor_Parking_Zone_22,31.0,2024-05-04,12.3,12.5,2024-05-04T06:06:12,2024-05-04T20:04:13,"Rain, Overcast",1st & M St NE,classic,15.0,42.0,DCA-CABI,4th & M St SW,classic,22.0,42.0,DCA-CABI,1st & M St NE,4th & M St SW,-77.005483,38.905697,-77.017807,38.876941,0,0,2024,5,4,Saturday,1.00
3,0F7347CF6409CEB7,classic_bike,2024-05-04 14:49:59,2024-05-04 14:54:11,Jefferson Memorial,31249,Hains Point/Buckeye & Ohio Dr SW,31273,casual,4.200000,POINT (-77.037413 38.879819),POINT (-77.03023 38.878433),ANC 2A,Res_Visitor_Parking_Zone_3,25.0,ANC 6D,Res_Visitor_Parking_Zone_22,31.0,2024-05-04,12.3,12.5,2024-05-04T06:06:12,2024-05-04T20:04:13,"Rain, Overcast",Jefferson Memorial,classic,23.0,42.0,DCA-CABI,Hains Point/Buckeye & Ohio Dr SW,classic,17.0,42.0,DCA-CABI,Jefferson Memorial,Hains Point/Buckeye & Ohio Dr SW,-77.037413,38.879819,-77.030230,38.878433,0,1,2024,5,4,Saturday,1.50
4,2022D12F38B197C3,classic_bike,2024-05-04 19:23:20,2024-05-04 19:41:35,New Hampshire Ave & T St NW,31229,14th & G St NW,31238,casual,18.250000,POINT (-77.038252 38.915544),POINT (-77.03189760446548 38.89816396413188),ANC 2B,Res_Visitor_Parking_Zone_4,20.0,ANC 2C,Res_Visitor_Parking_Zone_5,27.0,2024-05-04,12.3,12.5,2024-05-04T06:06:12,2024-05-04T20:04:13,"Rain, Overcast",New Hampshire Ave & T St NW,classic,23.0,42.0,DCA-CABI,14th & G St NW,classic,23.0,42.0,DCA-CABI,New Hampshire Ave & T St NW,14th & G St NW,-77.038252,38.915544,-77.031898,38.898164,0,0,2024,5,4,Saturday,1.00


Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'member_casual', 'duration_min', 'geometry_start',
       'geometry_end', 'Zone_start_zone_name', 'Zone_start_GIS_ID',
       'Zone_start_OBJECTID', 'Zone_end_zone_name', 'Zone_end_GIS_ID',
       'Zone_end_OBJECTID', 'date', 'Weather_temp', 'Weather_windspeedmean',
       'Weather_sunrise', 'Weather_sunset', 'Weather_conditions',
       'Bikeshare_NAME_start', 'Bikeshare_STATION_TYPE_start',
       'Bikeshare_CAPACITY_start', 'Bikeshare_REGION_ID_start',
       'Bikeshare_REGION_NAME_start', 'Bikeshare_NAME_end',
       'Bikeshare_STATION_TYPE_end', 'Bikeshare_CAPACITY_end',
       'Bikeshare_REGION_ID_end', 'Bikeshare_REGION_NAME_end',
       'StartStation_NAME', 'EndStation_NAME', 'start_longitude',
       'start_latitude', 'end_longitude', 'end_latitude',
       'start_near_any_mall', 'end_near_any_mall', 'year', 'month', 'day',


b.4

Station Capacity Classification

In [124]:
def assign_size_category(cap):
    if cap <= low_threshold:
        return 'Small'
    elif cap <= high_threshold:
        return 'Medium'
    else:
        return 'Large'

In [125]:
start_stations = merged_data[['start_station_id', 'Bikeshare_CAPACITY_start']].rename(
    columns={'start_station_id': 'station_id', 'Bikeshare_CAPACITY_start': 'capacity'}
)

end_stations = merged_data[['end_station_id', 'Bikeshare_CAPACITY_end']].rename(
    columns={'end_station_id': 'station_id', 'Bikeshare_CAPACITY_end': 'capacity'}
)

# Combine both into one list of stations
stations_list = pd.concat([start_stations, end_stations])


# Remove nulls and duplicates (in case some stations appear twice)
stations_list = stations_list.dropna(subset=['capacity'])
stations_list = stations_list.drop_duplicates(subset='station_id')

low_threshold = np.percentile(stations_list['capacity'], 33)
high_threshold = np.percentile(stations_list['capacity'], 66)

stations_list['station_size'] = stations_list['capacity'].apply(assign_size_category)

merged_data = merged_data.drop(columns=['start_station_size', 'end_station_size'], errors='ignore')


# Step 4: Apply the function to create the new column
stations_list['station_size'] = stations_list['capacity'].apply(assign_size_category)

merged_data = merged_data.merge(
    stations_list[['station_id', 'station_size']],
    left_on='start_station_id',
    right_on='station_id',
    how='left'
).rename(columns={'station_size': 'start_station_size'}).drop(columns=['station_id'])

# Merge end station sizes
merged_data = merged_data.merge(
    stations_list[['station_id', 'station_size']],
    left_on='end_station_id',
    right_on='station_id',
    how='left'
).rename(columns={'station_size': 'end_station_size'}).drop(columns=['station_id'])


# Now `stations` has: station_id, capacity, and station_size
print(stations_list.head())

   station_id  capacity station_size
0       31238      23.0        Large
1       31247      23.0        Large
2       31603      15.0        Small
3       31249      23.0        Large
4       31229      23.0        Large


In [126]:
stations_list['station_size'].value_counts()

station_size
Small     419
Medium    262
Large     119
Name: count, dtype: int64

b.5

Calculate Nearest Stop Distances

Bus Stops Data Cleaning and Preparation

In [127]:
# 1. Load the bus stops data
bus_stops = pd.read_csv("datasets/Shuttle_Bus_Stops.csv")
print(f"[Step 1] Loaded bus stops: {len(bus_stops)} rows")

# 2. Remove duplicates by location (LATITUDE, LONGITUDE)
before = len(bus_stops)
bus_stops = bus_stops.drop_duplicates(subset=['LATITUDE', 'LONGITUDE'])
after = len(bus_stops)
print(f"[Step 2] Duplicates removed: {before - after} | Rows after: {after}")

# 3. Remove rows with missing coordinates
critical_cols = ['LATITUDE', 'LONGITUDE']
before = len(bus_stops)
bus_stops = bus_stops.dropna(subset=critical_cols)
after = len(bus_stops)
print(f"[Step 3] Removed rows with missing coordinates: {before - after} | Rows after: {after}")

# 4. Create geometry column (as Points)
bus_stops['geometry'] = bus_stops.apply(lambda row: Point(row['LONGITUDE'], row['LATITUDE']), axis=1)
print("[Step 4] Geometry column created.")

# 5. Convert to GeoDataFrame and set CRS
gdf_bus_stops = gpd.GeoDataFrame(bus_stops, geometry='geometry', crs="EPSG:4326")
print("[Step 5] Converted to GeoDataFrame with CRS=EPSG:4326.")

# 6. Keep only coordinates and geometry
gdf_bus_stops = gdf_bus_stops[['LATITUDE', 'LONGITUDE', 'geometry']]
print("[Step 6] Selected columns: LATITUDE, LONGITUDE, geometry.")

# 7. Show sample of cleaned data
print(gdf_bus_stops.head())

[Step 1] Loaded bus stops: 102 rows
[Step 2] Duplicates removed: 6 | Rows after: 96
[Step 3] Removed rows with missing coordinates: 0 | Rows after: 96
[Step 4] Geometry column created.
[Step 5] Converted to GeoDataFrame with CRS=EPSG:4326.
[Step 6] Selected columns: LATITUDE, LONGITUDE, geometry.
    LATITUDE  LONGITUDE                    geometry
0  38.958828 -77.085372  POINT (-77.08537 38.95883)
1  38.952201 -77.002336   POINT (-77.00234 38.9522)
2  38.948072 -77.079460  POINT (-77.07946 38.94807)
3  38.947901 -77.078521   POINT (-77.07852 38.9479)
4  38.945924 -77.079102   POINT (-77.0791 38.94592)


Metro Stops Data Cleaning and Transformation

In [128]:

# 1. Load metro stops data
metro = pd.read_csv("datasets/Metro_Bus_Stops.csv")
print(f"[Step 1] Loaded metro_stops: {len(metro)} rows")

# 2. Remove duplicates by coordinates (X, Y)
before = len(metro)
metro = metro.drop_duplicates(subset=['X', 'Y'])
after = len(metro)
print(f"[Step 2] Duplicates removed: {before - after} | Rows after: {after}")

# 3. Remove rows with missing coordinates
critical_cols = ['X', 'Y']
before = len(metro)
metro = metro.dropna(subset=critical_cols)
after = len(metro)
print(f"[Step 3] Removed rows with missing coordinates: {before - after} | Rows after: {after}")

# 4. Create geometry column (Points)
metro['geometry'] = metro.apply(lambda row: Point(row['X'], row['Y']), axis=1)
print("[Step 4] Geometry column created.")

# 5. Convert to GeoDataFrame, CRS assumed to be EPSG:3857 (Web Mercator)
gdf_metro = gpd.GeoDataFrame(metro, geometry='geometry', crs="EPSG:3857")
print("[Step 5] Converted to GeoDataFrame with CRS=EPSG:3857.")

# 6. Transform to WGS84 (EPSG:4326) for consistency with 
gdf_metro = gdf_metro.to_crs("EPSG:4326")
print("[Step 6] Reprojected to CRS=EPSG:4326.")

# 7. Keep only geometry
gdf_metro = gdf_metro[['geometry']]
print("[Step 7] Selected geometry column only.")

# 8. Show sample
print(gdf_metro.head())


[Step 1] Loaded metro_stops: 10044 rows
[Step 2] Duplicates removed: 0 | Rows after: 10044
[Step 3] Removed rows with missing coordinates: 0 | Rows after: 10044
[Step 4] Geometry column created.
[Step 5] Converted to GeoDataFrame with CRS=EPSG:3857.
[Step 6] Reprojected to CRS=EPSG:4326.
[Step 7] Selected geometry column only.
                     geometry
0  POINT (-77.08676 38.95911)
1  POINT (-76.72056 38.94844)
2  POINT (-77.17547 38.83658)
3  POINT (-77.05237 38.72475)
4  POINT (-77.06253 38.71265)


In [134]:
def geos_to_array(gseries):
    # Converts a GeoSeries of Points to Nx2 numpy array [lng, lat]
    return np.array([(pt.x, pt.y) for pt in gseries])
bus_coords = geos_to_array(gdf_bus_stops['geometry'])
metro_coords = geos_to_array(gdf_metro['geometry'])
bus_tree = cKDTree(bus_coords)
metro_tree = cKDTree(metro_coords)
# ---- 5. Prepare coordinates for start and end of each trip ----
start_coords = merged_data['geometry_start'].apply(lambda p: (p.x, p.y)).tolist()
assert np.isfinite(start_coords).all()

end_coords = merged_data['geometry_end'].apply(lambda p: (p.x, p.y)).tolist()
assert np.isfinite(end_coords).all()


# ---- 6. Query nearest bus stop (start & end) ----
_, bus_idx_start = bus_tree.query(start_coords, k=1)
_, bus_idx_end   = bus_tree.query(end_coords, k=1)


# ---- 7. Query nearest metro stop (start & end) ----
_, metro_idx_start = metro_tree.query(start_coords, k=1)
_, metro_idx_end   = metro_tree.query(end_coords, k=1)

# ---- 8. Calculate distance in meters using haversine formula ----

# Bus stops: get coordinates
bus_lats = gdf_bus_stops['geometry'].y.values
bus_lngs = gdf_bus_stops['geometry'].x.values

metro_lats = gdf_metro['geometry'].y.values
metro_lngs = gdf_metro['geometry'].x.values

merged_data['nearest_bus_stop_lat_start']  = bus_lats[bus_idx_start]
merged_data['nearest_bus_stop_lng_start']  = bus_lngs[bus_idx_start]
merged_data['bus_stop_dist_start'] = [
    hf.haversine_np(lon1, lat1, lon2, lat2)
    for (lon1, lat1, lon2, lat2) in zip(
        [pt[0] for pt in start_coords],
        [pt[1] for pt in start_coords],
        bus_lngs[bus_idx_start],
        bus_lats[bus_idx_start]
    )
]
merged_data['nearest_bus_stop_lat_end'] = bus_lats[bus_idx_end]
merged_data['nearest_bus_stop_lng_end'] = bus_lngs[bus_idx_end]
merged_data['bus_stop_dist_end'] = [
    hf.haversine_np(lon1, lat1, lon2, lat2)
    for (lon1, lat1, lon2, lat2) in zip(
        [pt[0] for pt in end_coords],
        [pt[1] for pt in end_coords],
        bus_lngs[bus_idx_end],
        bus_lats[bus_idx_end]
    )
]

merged_data['nearest_metro_stop_lat_start'] = metro_lats[metro_idx_start]
merged_data['nearest_metro_stop_lng_start'] = metro_lngs[metro_idx_start]
merged_data['metro_stop_dist_start'] = [
    hf.haversine_np(lon1, lat1, lon2, lat2)
    for (lon1, lat1, lon2, lat2) in zip(
        [pt[0] for pt in start_coords],
        [pt[1] for pt in start_coords],
        metro_lngs[metro_idx_start],
        metro_lats[metro_idx_start]
    )
]
merged_data['nearest_metro_stop_lat_end'] = metro_lats[metro_idx_end]
merged_data['nearest_metro_stop_lng_end'] = metro_lngs[metro_idx_end]
merged_data['metro_stop_dist_end'] = [
    hf.haversine_np(lon1, lat1, lon2, lat2)
    for (lon1, lat1, lon2, lat2) in zip(
        [pt[0] for pt in end_coords],
        [pt[1] for pt in end_coords],
        metro_lngs[metro_idx_end],
        metro_lats[metro_idx_end]
    )
]

AssertionError: 

b.6

Indicator for Trips Starting or Ending in the Main Commercial Area

Already done above

b.7

Calculating Distance to the Main Commercial Area

In [136]:
merged_data['end_distance_to_closest_mall_m'] = merged_data.apply(
    lambda row: hf.distance_to_closest_mall(row['end_latitude'], row['end_longitude'], shopping_centers),
    axis=1
)

b.8

Daily Trip Location Aggregation Using Geographic Hashing

In [137]:
precision = 6

merged_data['start_geohash'] = merged_data.apply(
    lambda row: pgh.encode(row['start_latitude'], row['start_longitude'], precision=precision)
    if not np.isnan(row['start_latitude']) and not np.isnan(row['start_longitude']) else np.nan,
    axis=1
)

merged_data['end_geohash'] = merged_data.apply(
    lambda row: pgh.encode(row['end_latitude'], row['end_longitude'], precision=precision)
    if not np.isnan(row['end_latitude']) and not np.isnan(row['end_longitude']) else np.nan,
    axis=1
)

In [144]:
merged_data['start_geohash'].value_counts()

start_geohash
dqcjqg    57646
dqcjr5    53565
dqcjrh    48275
dqcjr1    42152
dqcjr4    37363
          ...  
dqcq07       13
dqcm4v       10
dqcq05       10
dqcm91        3
dqcm3p        2
Name: count, Length: 426, dtype: int64

b.9

Classification of Geographic Sectors by Daily Trip Frequency

In [145]:
daily_trip_counts = merged_data.groupby(['start_geohash', 'date']).size().reset_index(name='trip_count')

avg_trips_per_geohash = daily_trip_counts.groupby('start_geohash')['trip_count'].mean().reset_index()
avg_trips_per_geohash.rename(columns={'trip_count': 'avg_daily_trips'}, inplace=True)

quantiles = avg_trips_per_geohash['avg_daily_trips'].quantile([0.22, 0.88])
low_thresh = quantiles[0.22]
high_thresh = quantiles[0.88]

In [147]:
avg_trips_per_geohash

,start_geohash,avg_daily_trips
0,dqbvrz,1.186047
1,dqbvx3,1.218750
2,dqbvx7,1.216981
3,dqbvx9,1.108696
4,dqbvxb,1.166667
...,...,...
421,dqcq0h,1.000000
422,dqcq0k,1.066667
423,dqcq31,1.129032
424,dqcq34,1.228571


b.10

Weather Condition Binning into Three Categories

In [148]:
def classify_weather(condition):
    if pd.isna(condition):
        return np.nan
    condition = condition.lower()
    if any(w in condition for w in ['clear', 'sunny']):
        return 'Sunny'
    elif any(w in condition for w in ['rain', 'storm', 'shower', 'drizzle']):
        return 'Rainy'
    elif any(w in condition for w in ['cloud', 'overcast', 'fog', 'mist']):
        return 'Cloudy'
    else:
        return 'Cloudy'

In [150]:
merged_data['weather_category'] = merged_data['Weather_conditions'].apply(classify_weather)

# Step 3: Drop old weather columns
merged_data.drop(columns=[
    'Weather_temp',
    'Weather_windspeedmean',
    'Weather_sunrise',
    'Weather_sunset',
    'Weather_conditions'
], inplace=True)

b.11

Building a Daily Revenue Time Series with Weather Data

In [152]:
merged_data['ended_at'] = pd.to_datetime(merged_data['ended_at'])

merged_data['payment_day'] = merged_data['ended_at'].dt.date

daily_revenue = merged_data.groupby(['payment_day', 'weather_category'])['trip_price'].sum().reset_index()

long_format = daily_revenue.copy()

wide_format = daily_revenue.pivot(index='payment_day', columns='weather_category', values='trip_price').fillna(0).reset_index()
wide_format.columns.name = None  # remove pivoted column name

print("Long Format:")
print(long_format.head())

print("\nWide Format:")
print(wide_format.head())

Long Format:
  payment_day weather_category  trip_price
0  2024-01-01            Rainy     4476.62
1  2024-01-02           Cloudy     7815.20
2  2024-01-02            Rainy      810.98
3  2024-01-03           Cloudy     8901.43
4  2024-01-04           Cloudy     8721.21

Wide Format:
  payment_day   Cloudy    Rainy  Sunny
0  2024-01-01     0.00  4476.62    0.0
1  2024-01-02  7815.20   810.98    0.0
2  2024-01-03  8901.43     0.00    0.0
3  2024-01-04  8721.21     0.00    0.0
4  2024-01-05  8539.16     0.00    0.0


b.12

Add a Custom Feature

In [154]:
# 1. Bar plot showing statistics of the top 5 start stations that had most trips
top5_starts = merged_data['start_station_name'].value_counts().nlargest(5).reset_index()
top5_starts.columns = ['start_station_name', 'trip_count']
fig1 = px.bar(
    top5_starts,
    x='start_station_name',
    y='trip_count',
    title='Top 5 Start Stations by Trip Count',
    labels={'trip_count': 'Number of Trips', 'start_station_name': 'Start Station'},
    color='trip_count',
    color_continuous_scale='Blues'
)

fig1.show()

In [156]:
# 2. Bar plot: distribution of trips by bike and subscription type
bike_subs_dist = merged_data.groupby(['rideable_type', 'member_casual']).size().reset_index(name='trip_count')
fig2 = px.bar(
    bike_subs_dist,
    x='rideable_type',
    y='trip_count',
    color='member_casual',
    barmode='group',
    title='Trips by Bike Type and Subscription Type',
    labels={'rideable_type': 'Bike Type', 'trip_count': 'Number of Trips', 'member_casual': 'User Type'}
)

fig2.show()

In [157]:
top5_names = top5_starts['start_station_name'].tolist()
sunburst_df = merged_data[merged_data['start_station_name'].isin(top5_names)]
sunburst_data = sunburst_df.groupby(
    ['start_station_name', 'rideable_type', 'member_casual']
).size().reset_index(name='trip_count')

fig3 = px.sunburst(
    sunburst_data,
    path=['start_station_name', 'rideable_type', 'member_casual'],
    values='trip_count',
    title='Trip Distribution for Top 5 Start Stations'
)

fig3.show()

In [ ]:
# 4a. Histogram: station size distribution
fig4a = px.histogram(
    merged_data,
    x='start_station_size',
    color='start_station_size',
    title='Distribution of Start Station Sizes',
    labels={'start_station_size': 'Station Size'}
)

fig4a.show()

In [ ]:
fig5 = px.histogram(
    merged_data,
    x="duration_min",
    nbins=100,
    title="Distribution of Trip Duration (Minutes)",
    labels={"duration_min": "Duration (minutes)"}
)
fig5.update_layout(bargap=0.2)

In [ ]:
fig6 = px.box(
    merged_data,
    x="rideable_type",
    y="duration_min",
    title="Trip Duration by Bike Type (Log Scale)",
    labels={"rideable_type": "Bike Type", "duration_min": "Duration (minutes)"},
    log_y=True
)
fig6.show()

In [ ]:
fig7 = px.box(
    merged_data,
    x="member_casual",
    y="duration_min",
    title="Trip Duration by Subscription Type",
    labels={"member_casual": "User Type", "duration_min": "Duration (minutes)"},
    log_y=True
)


fig7.show()

In [162]:
import folium
from folium.plugins import MarkerCluster

# 1. Check if any trips > 24 hours exist
long_trips = merged_data[merged_data['duration_min'] > 25 ].copy()
print(f"Found {len(long_trips)} long trips (>24h)")

if len(long_trips) == 0:
    print("No trips exceed 24 hours. Adjust your filter or check data.")
else:
    # 2. Use correct column names (adjust based on your actual columns!)
    start_trips = long_trips[['start_station_name', 'start_latitude', 'start_longitude']]
    end_trips = long_trips[['end_station_name', 'end_latitude', 'end_longitude']]

    # Standardize column names
    start_trips.columns = ['station_name', 'lat', 'lon']
    end_trips.columns = ['station_name', 'lat', 'lon']

    # 3. Combine and drop invalid coordinates
    stations_all = pd.concat([start_trips, end_trips]).dropna(subset=['lat', 'lon'])
    print(f"Stations with valid coords: {len(stations_all)}")

    if len(stations_all) > 0:
        # Count trips per station
        station_counts = stations_all.groupby(['station_name', 'lat', 'lon']).size().reset_index(name='trip_count')
        
        # Filter to DC area (optional)
        station_counts = station_counts[
            (station_counts['lat'].between(38.80, 38.97)) & 
            (station_counts['lon'].between(-77.12, -76.90))
        ]
        print("Top stations:\n", station_counts.head())

        # Create map only if data exists
        m = folium.Map(location=[38.9072, -77.0369], zoom_start=13)
        for _, row in station_counts.iterrows():
            folium.CircleMarker(
                location=[row['lat'], row['lon']],
                radius=row['trip_count'] * 0.5,
                color='red',
                fill=True,
                popup=f"{row['station_name']}: {row['trip_count']} long trips"
            ).add_to(m)
        display(m)  # For Jupyter; use `m.save('map.html')` otherwise
    else:
        print("No stations with valid coordinates found.")

Found 263323 long trips (>24h)
Stations with valid coords: 448376
Top stations:
               station_name        lat        lon  trip_count
0   S Scott St & 13th St S  38.862862 -77.077160         123
1           10th & E St NW  38.895914 -77.026064        1226
2           10th & G St NW  38.898243 -77.026235         679
3           10th & H St NE  38.899984 -76.992725         461
4           10th & K St NW  38.902400 -77.026220        1163
